In [4]:
import numpy as np
import scipy.linalg as la
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Statevector

In [ ]:
def create_a(N=4):
    a = np.zeros((N, N), dtype=complex)
    for n in range(N - 1):
        a[n, n + 1] = np.sqrt(n + 1)
    return a, a.T.conj()

def create_x(N=4):
    a, adag = create_a(N)
    return (a + adag) / np.sqrt(2.0)

def create_H0(N=4):
    # H0 = a^\dag a + 1/2 I  => diag(0,1,2,3) + 1/2
    return np.diag(np.arange(N, dtype=float)) + 0.5 * np.eye(N)

In [6]:
x2_coeffs = coeffs_by_label(x2_op, LABELS)
x4_coeffs = coeffs_by_label(x4_op, LABELS)

In [5]:
N = 4
lam_target = 1

x  = create_x(N)
x2 = x @ x
x4 = x2 @ x2
H0 = create_H0(N)

# ------------------------------
# 2) Pauli decompositions
# ------------------------------
# canonical little-endian labels; rightmost char -> qubit 0
LABELS = ['II', 'XI', 'XZ', 'ZI', 'ZZ']

def coeffs_by_label(op: SparsePauliOp, labels=LABELS):
    m = {str(p): float(c.real) for p, c in zip(op.paulis, op.coeffs)}
    return np.array([m.get(l, 0.0) for l in labels], dtype=float)

x2_op = SparsePauliOp.from_operator(x2)
x4_op = SparsePauliOp.from_operator(x4)
x2_coeffs = coeffs_by_label(x2_op, LABELS)
x4_coeffs = coeffs_by_label(x4_op, LABELS)

# ------------------------------
# 3) Exact target eigenpairs for H0 + λ x^4 (no CT)
# ------------------------------
H_lam = H0 + lam_target * x4
evals_f, evecs_f = la.eigh(H_lam)
ordF = np.argsort(evals_f)
v0f = evecs_f[:, ordF[0]]
v1f = evecs_f[:, ordF[1]]

# Start states: eigenvectors of H0
eval0, evec0 = la.eigh(H0)
ord0 = np.argsort(eval0)
g0, g1 = evec0[:, ord0[0]], evec0[:, ord0[1]]

# ------------------------------
# 4) Schedules (two-stage, ramp λ on, CT on; hold λ, CT off)
# ------------------------------
def r(u):                     # smoothstep 3u^2 - 2u^3
    return u*u*(3.0 - 2.0*u)

def lamOf(u, lam=lam_target):
    if 0.0 <= u <= 1.0:
        return lam * r(u)
    elif 1.0 < u <= 2.0:
        return lam
    return 0.0

def dm2Of(u, lam=lam_target):
    if 0.0 <= u <= 1.0:
        return -6.0 * lam * r(u)                 # turn CT on
    elif 1.0 < u <= 2.0:
        return -6.0 * lam * (1.0 - r(u - 1.0))   # then turn CT off
    return 0.0

# ------------------------------
# 5) One Strang-split *gate* slice
#     H = H0 + λ x^4 + (½ δm²) x^2
#     H0 = 2*II - 0.5*Z0 - 1.0*Z1  (we skip the II global phase)
# ------------------------------
def strang_slice(qc: QuantumCircuit, lam_t: float, half_dm2_t: float, dt: float):
    # fuse x4 and x2 coefficients for this time
    cII, cXI, cXZ, cZ1, cZZ = lam_t * x4_coeffs + half_dm2_t * x2_coeffs
    # add H0 Z parts
    cZ0 = -0.5
    cZ1 += -1.0
    h = 0.5 * dt

    # half Z-family
    qc.rz(2 * cZ1 * h, 1)
    qc.rzz(2 * cZZ * h, 1, 0)
    qc.rz(2 * cZ0 * h, 0)

    # full X-family
    qc.rx(2 * cXI * dt, 1)
    qc.h(1); qc.rzz(2 * cXZ * dt, 1, 0); qc.h(1)

    # half Z-family
    qc.rz(2 * cZ1 * h, 1)
    qc.rzz(2 * cZZ * h, 1, 0)
    qc.rz(2 * cZ0 * h, 0)

# ------------------------------
# 6) Build the full circuit for the two-stage schedule
# ------------------------------
def build_circuit(lam=lam_target, tPrep=25.0, nSteps=250):
    qc = QuantumCircuit(2)
    dt = tPrep / (2.0 * nSteps)
    for j in range(1, 2 * nSteps + 1):
        u = (j - 0.5) / nSteps
        lam_t  = lamOf(u, lam)
        halfdm = 0.5 * dm2Of(u, lam)
        strang_slice(qc, lam_t, halfdm, dt)
    return qc

qc = build_circuit(lam_target, 25.0, 250)

# ------------------------------
# 7) Evolve states and compute fidelities (no global-phase dependence)
# ------------------------------
def fidelity(psi, phi):
    return float(abs(np.vdot(phi, psi))**2)

# Evolve |g0>, |g1> through the *circuit* (correct way; no unitary-vs-state confusion)
psi0 = Statevector(g0).evolve(qc).data
psi1 = Statevector(g1).evolve(qc).data
psi0 /= la.norm(psi0); psi1 /= la.norm(psi1)

F0 = fidelity(psi0, v0f)
F1 = fidelity(psi1, v1f)

print("Vacuum fidelity   |<E0(λ)|ψ>|^2 =", F0)
print("1-particle fidelity|<E1(λ)|ψ>|^2 =", F1)


Vacuum fidelity   |<E0(λ)|ψ>|^2 = 0.9997708567958397
1-particle fidelity|<E1(λ)|ψ>|^2 = 0.9996336166917964
